# 16 Dynamic Programming II - Applications

## Plan for the Lecture

1. Origins of DP - Richard Bellman's principle of optimality

2. DP optimisation strategies - Memoization vs tabulation / bottom-up vs top-down

3. Fibonacci exercise 

4. Network Flow - capacity - minimax and maximin

## Recap on runtimes

* Quadratic Time $O(n^2)$ : A specific type of polynomial time with performance scaling as the square of the input size. Common in simple, often inefficient algorithms.

* Polynomial Time $O(n^k)$ : A broad category encompassing all algorithms whose running times are polynomial in the input size. Central to the concept of efficient, tractable algorithms.

* Pseudo-Polynomial Time $O(n \cdot m)$ : Algorithms whose running time depends on the numeric values within the input rather than solely on input size. Efficient for small numeric inputs but can be impractical for large values.


## Scenario Exercise - Blackjack (21)

![blackjack_21](https://odentennis.wordpress.com/wp-content/uploads/2022/02/blackjack.gif)


* This simulation will allow a user to play Blackjack against a dealer. The game will handle:

    * Shuffling and dealing cards.
    * Calculating hand values, considering the flexible value of Aces.
    * Allowing the player to choose actions: Hit or Stand.
    * Implementing the dealer’s fixed strategy: hitting until reaching at least 17.
    * Determining the outcome of each round.


## Let's build a simple simulation of 21/Blackjack

In [11]:
import random

In [12]:
cards = ['Ace of Hearts', '2 of Diamonds', 'King of Clubs', '7 of Spades']
print("Original deck:", cards)

random.shuffle(cards) # Shuffle the deck
print("Shuffled deck:", cards)

Original deck: ['Ace of Hearts', '2 of Diamonds', 'King of Clubs', '7 of Spades']
Shuffled deck: ['2 of Diamonds', '7 of Spades', 'Ace of Hearts', 'King of Clubs']


In [13]:
class Card:
    """Represents a single playing card."""
    def __init__(self, suit, rank):
        self.suit = suit  # 'Hearts', 'Diamonds', 'Clubs', 'Spades'
        self.rank = rank  # '2' - '10', 'J', 'Q', 'K', 'A'

    def __str__(self):
        return f"{self.rank} of {self.suit}"

class Deck:
    """Represents a deck of 52 playing cards."""
    suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
    ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10',
             'J', 'Q', 'K', 'A']

    def __init__(self):
        self.cards = [Card(suit, rank) for suit in self.suits for rank in self.ranks]
        self.shuffle()

    def shuffle(self):
        """Shuffles the deck in place."""
        random.shuffle(self.cards)

    def deal_card(self):
        """Deals the top card from the deck."""
        if len(self.cards) == 0:
            raise ValueError("All cards have been dealt")
        return self.cards.pop()

    def __str__(self):
        """Returns a string representation of the deck."""
        return ', '.join(str(card) for card in self.cards)

In [14]:
deck = Deck() # shuffle the deck
print("Shuffled Deck:")
print(deck)

Shuffled Deck:
6 of Diamonds, A of Clubs, 4 of Hearts, 2 of Diamonds, 10 of Hearts, 7 of Diamonds, 5 of Clubs, 8 of Hearts, 3 of Diamonds, J of Hearts, K of Hearts, 8 of Spades, 3 of Spades, 2 of Spades, K of Clubs, K of Spades, 2 of Clubs, A of Spades, 6 of Spades, 5 of Spades, A of Diamonds, 6 of Clubs, J of Clubs, 10 of Diamonds, Q of Clubs, 10 of Spades, 5 of Hearts, 3 of Hearts, K of Diamonds, 9 of Diamonds, 5 of Diamonds, Q of Spades, 4 of Diamonds, 3 of Clubs, 10 of Clubs, 2 of Hearts, Q of Diamonds, 7 of Hearts, 8 of Diamonds, 7 of Spades, A of Hearts, 9 of Hearts, 4 of Spades, J of Spades, 6 of Hearts, 9 of Spades, 9 of Clubs, J of Diamonds, 4 of Clubs, 7 of Clubs, 8 of Clubs, Q of Hearts


In [17]:
deck = Deck() # shuffle the deck

# Deal first two cards?
print("\nDeal two cards:")
for _ in range(2):
    card = deck.deal_card()
    print(card)

while True:
    print("\nStick (s) or Twist (t)?")
    ans = input("Stick (s) or Twist (t)?")
    if ans.lower() == "t" :
        card = deck.deal_card()
        print(card)
    else:
        break



Deal two cards:
4 of Hearts
Q of Hearts

Stick (s) or Twist (t)?
6 of Diamonds

Stick (s) or Twist (t)?


## Exercise - warm up on this example

To warm up, amend the code above to count the picture cards (J, Q, K) as ten, and give the choice of whether ace is 11 or 1. 


## Can we apply Dynamic Programming to 21/Blackjack?





* `States (S)`: Represent the current situation in the game, typically defined by:
    * Player’s Hand Value: Sum of the player’s cards (considering soft and hard totals due to Aces).
    * Dealer’s Visible Card: The dealer’s upcard.
    * Additional Factors: Such as whether the player has a usable Ace, the number of decks in play, etc.

* `Actions (A)`: Possible decisions the player can make, e.g., hit, stand, double down, split.

* `Transition Probabilities (P)`: The likelihood of moving from one state to another based on the chosen action and the randomness of card drawing.

* `Rewards (R)`: The outcome of actions, typically:
    * Win: Positive reward.
    * Lose: Negative reward.
    * Push (Tie): Neutral reward.


## Scenario Exercise - Super-Mario 

![super_mario](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQC3rfJrKWyXB02JJADPgFRtXKGbabn0GMq4g&s)

![super_mario_bros_1_nes](https://miro.medium.com/v2/resize:fit:1002/1*7TLBg5I9DSrvVwebZoA6JQ.gif)

Dynamic programming (DP) can be applied to Super Mario to solve complex problems by breaking them down into overlapping subproblems. In a game like Mario, DP can be used for:

1.	Pathfinding: Finding the shortest or optimal path in levels with obstacles.

2.	Coin Collection: Maximizing the number of coins collected by choosing the best route.

3.	Enemy Avoidance: Minimizing risk by finding paths with fewer or weaker enemies.

4.	Lives and Power-Ups Management: Optimally deciding when to collect power-ups or take risks for extra lives.

This approach involves storing and reusing solutions to subproblems to make the gameplay more efficient.

1. Pathfinding:

	* Break down each level into sub-levels (e.g., checkpoints or blocks).
	* Define a cost function for each block (e.g., time, obstacles).
	* Use DP to compute the minimum cost from the start to the finish, storing subproblem results for reuse.

2. Coin Collection:

	* Assign values to blocks containing coins.
	* Use DP to calculate the maximum coins collected along any possible path.

3. Enemy Avoidance:

	* Treat enemies as obstacles and define the risk (penalty) associated with each block.
	* Use DP to minimize the risk by evaluating all possible paths.

4. Lives and Power-Ups Management:

	* Store Mario’s current state (e.g., power-ups and lives).
	* Use DP to determine optimal power-up collection strategies by maximizing future gains.
	
Starting with a grid or graph representation of the game level will help lay the groundwork for each of these problems.

## MIT 6006 analysis of Super Mario Bros (NES 1993)

Goal: Run through and maximise score or minimise time lost

* Given level $n$ information
* Small screen $w$ x $h$ (320p)
* Configuration - Game state ($S$): every pixel / square on screen: $c^{w\cdot h}$
    * everything on screen 
    * Mario's Velocity 
* Score ($S$) 
* Time ($T$)
* Screen vs Level 

$O(w \cdot c^{w\cdot h} \cdot S \cdot T) $




* Can draw a graph for all configurations of the above

* For every configration, what are the possible things you could do? 
    * Constant number of choice 

DP: all of the above are subproblems.

Can relate this subproblem to a constant number of other subproblems.

You only pay constant time per subproblem. $O(1)$

Psuedo-polynomial with respect to $S$ and $T$


## Scenario Exercise - Stock Management

## Summary 

* a

* a

* a

## Exercise 

Floyd Warshall algorithm - all-pairs shortest paths
* This is a dynamic programming approach to build up the solution for the shortest paths step by step. 
* It considers each pair of vertices and iteratively improves the path between them by considering each possible intermediate vertex.
*
* Step 1. Initialization: 
*   Create a 2D array dist where dist[i][j] represents the shortest distance from vertex i to vertex j. 
*   Initialize dist[i][j] to the weight of the edge from i to j if it exists, otherwise to infinity. 
*   Set dist[i][i] = 0 for all vertices i.
*
* Step 2. Dynamic Programming:  
*   Update the distance array dist by considering each vertex as an intermediate vertex and 
*   updating the shortest paths accordingly. For each pair of vertices (i, j), update dist[i][j].
*
* Step 3. Result: 
*   After considering all vertices as intermediate vertices, the dist array contains the 
*   shortest paths between all pairs of vertices.

## Exercise 

Write the Bellman-Ford algorithm with a dynamic programming table.

In [ ]:
def bellman_ford_dp(vertices, edges, source):
    ...

In [2]:
V = 5

# Edges (u, v, weight)
edges = [
    (0, 1, -1),
    (0, 2, 4),
    (1, 2, 3),
    (1, 3, 2),
    (1, 4, 2),
    (3, 2, 5),
    (3, 1, 1),
    (4, 3, -3),
]

# Compute shortest paths from vertex 0
distances = bellman_ford_dp(V, edges, 0)

if distances:
    print("Vertex\tDistance from Source")
    for i, d in enumerate(distances):
        print(f"{i}\t{d}")

Vertex	Distance from Source
0	0
1	-1
2	2
3	-2
4	1


## Exercise 

Given a directed graph $G$ with $n$ nodes and $m$ edges, each edge $(u, v)$ has a capacity $c(u, v)$. Determine the maximum possible flow from a source $s$ to a sink $t$, such that the maximum flow through any single edge is minimized.

This problem minimizes the “bottleneck” of the network, ensuring that no single edge carries an excessive amount of the flow.

## Exericse

Given a flow network, find a path from $s$ to $t$ such that the minimum edge capacity along the path is maximized.

Given a directed graph $G$ with $n$ nodes and $m$ edges, find the maximum flow $F$ from source $s$ to sink $t$, such that the minimum flow through any edge in the solution is maximized.

Extension: This problem maximizes the smallest amount of flow on any edge used in the final flow, ensuring a “balanced” distribution of flow across the network.

## Additional exercises - Strings

## Exercise: 

Without using any functions provided by libraries (e.g. `reverse()`), implement a function that will reverse the contents of a string. 

In [ ]:
# Write your solution here or in a dedicated py file.


## Exercise: 

Write a function that checks if a given string is a palindrome (same word forwards as backwards). Examples of palindromes include 'eve', 'madam', 'racecar' etc.


In [ ]:
# Write your solution here or in a dedicated py file.


## Exercise:
Write a function to determine if a string has all unique characters. Return `True` or `False` respectively.

In [ ]:
# Write your solution here or in a dedicated py file.


## Exercise:

Write a function that will replace all spaces in a string with "%20". This is the hexedecimal encoding that is used to replace spaces in URLs (which cannot contain spaces). 

In [ ]:
# Write your solution here or in a dedicated py file.


## Exercise: 

Write a function that checks whether two given strings are anagrams of each other.
Definition of an anagram is being able to make another word from the same characters: 'mood' and 'doom', 'listen' and 'silent' 

In [ ]:
# Write your solution here or in a dedicated py file.


## Exercise: 

Write a function that will replace all spaces in a string with "%20". This is the hexedecimal encoding that is used to replace spaces in URLs (which cannot contain spaces). 

In [ ]:
# Write your solution here or in a dedicated py file.


## Additional Exercises - Maths

## Exercise 

Write to function which will evaluate whether a number passed in is prime (divisible by itself and 1) or not. Return `True` if the number is prime, and return `False` if the number is not. Test this with a range of positive integers. 

In [ ]:
# Write your solution here or in a dedicated py file.


## Exercise 

Generate three side lengths of a triangle (positive integers). Write a function that will determine whether these form a right-angled triangle using Pythagoras’ theorem.

As a reminder, Pythagoras' theorem is: $a^2 + b^2 = c^2$


In [ ]:
# Write your solution here or in a dedicated py file.


## Exercise 
Calculate the distance between two points $(x_1, y_1)$ and $(x_2, y_2)$ in a 2D plane using Pythagoras’ theorem.

Hint: To calculate distance (denoted as $d$) applying the following formula:   
$d = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}$ 

Extension: import matplotlib and draw these points and the line between them on a 2D plot.

In [ ]:
# Write your solution here or in a dedicated py file.


## Exercise 

Now, write a function which will output all prime numbers up to $N$ (which you pass in).
If you get stuck, research the 'Sieve of Eratosthenes` and implement this algorithm to output the prime numbers in sequence up until the upper bound $N$ as specified.

In [ ]:
# Write your solution here or in a dedicated py file.


## Additional OOP Exercises!

## Exercise 1: 

Design the data structures for a generic deck of cards. Explain how you would subclass the data structures to implement blackjack (21). 

In [ ]:
# Write your solution here or in a dedicated py file.


## Exercise 2: 

Imagine you have a call centre with three levels of employees: respondent, manager and director. An incoming telephone call must be first allocated to a respondent who is free. If a respondent can't handle the call, they must escalate the call to the manager. If the manager is not free or not able to handle it, then the call should be escalated to a director. Design classes and data structures for this problem. Implement a function `dispatch_call()` which assigns the call to the first available employee. 

In [ ]:
# Write your solution here or in a dedicated py file.


## Exercise 3: 

Implement a jigsaw puzzle. Design the data structures and explain an algorithm to solve the puzzle. You can assume that you have a `fits_with()` function, which when passed two puzzle pieces, returns `True` if the pieces fit together. 

In [ ]:
# Write your solution here or in a dedicated py file.
